<a href="https://colab.research.google.com/github/Yaraalfouzan/Algorithms-phase1/blob/main/RougeL_Labeling_of_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##QA pairs generated using llama8b on xquad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pandas as pd

file_path = '/content/drive/My Drive/generated_answers_Llama3.1-8b-xquad.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
print(df.head())

   question_id                                           question  \
0          320                             ما هو المرفق بالسواكن؟   
1         1007     ما الدليل الذي تقدمه الجينات المُتَبَرَّع بها؟   
2          605  ماذا حدث في عام 1992 ضمن إجراءٍ على نطاق الممل...   
3          425                   متى أحدِثت توجيهية مجلس الأعمال؟   
4            4  من سجل أعلى عدد من الاستحواذات للفريق هذا الموسم؟   

                                     original_answer  \
0  {'text': ['غلاف التوربين'], 'answer_start': [1...   
1  {'text': ['وجود البلاستيدات الخضراء المفقود'],...   
2  {'text': ['أصبحت فيه كليات الفنون التطبيقية جا...   
3  {'text': ['استشارات الطاقة العمالية في الأعمال...   
4    {'text': ['كاوان شورت'], 'answer_start': [216]}   

                                   generated_answers  
0  [المرفق بالسواكن هو أحد الأوتار التالية للشخص ...  
1  [تتضمن الدنا العِرضيّة نطاقات الدنا المُتِبَرِ...  
2  [حصل على جائزة نوبل في الآداب وتم إشهار قانون ...  
3  [6/ 13/ 1433 هـ, أجرى مجل

#Test RougeL

In [ ]:
from transformers import AutoTokenizer
from rouge_score import rouge_scorer

# Load tokenizer
model_name = "core42/jais-13b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], tokenizer=tokenizer)

# Example strings
pred_str = 'السلام عليكم كيف حالك'
label_str = 'السلام عليكم صديقي كيف حالك'

scores = scorer.score(label_str, pred_str)

for key in scores:
    print(f'{key}: {scores[key]}')

tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

rougeL: Score(precision=1.0, recall=0.8, fmeasure=0.888888888888889)


#Use RougeL on data

In [ ]:
import json

# Load tokenizer
model_name = "core42/jais-13b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# load data from JSON file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# save labels to a new JSON file
def save_labels(output_data, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(output_data, file, ensure_ascii=False, indent=4)

# calculate ROUGE score and check hallucination
def check_hallucination(data):
    output_data = []  #list to store the labeled data

    for item in data:
        question_id = item['question_id']
        question = item['question']
        original_answer = item['original_answer']['text'][0]  #ground truth

        # Dictionary to store results for each question
        question_data = {
            "question_id": question_id,
            "question": question,
            "original_answer": item['original_answer'],
            "generated_answers": []
        }

        for generated_answer in item['generated_answers']:
            #ROUGE-L F1 score
            scores = scorer.score(original_answer, generated_answer)
            rouge_l_f1 = scores['rougeL'].fmeasure

            # Check if hallucinated or not
            if rouge_l_f1 <= 0.3:
                status = "Hallucinated"
            else:
                status = "Non-Hallucinated"

            # Append the labeled answer with F1 score to the question data
            question_data["generated_answers"].append({
                "text": generated_answer,
                "f1_score": round(rouge_l_f1, 2),
                "label": status
            })

        output_data.append(question_data)

    return output_data

# Load data from JSON file
input_file_path = '/content/drive/My Drive/generated_answers_Llama3.1-8b-xquad.json'
data = load_data(input_file_path)

labeled_data = check_hallucination(data)

# Save the labeled data to a new JSON file
output_file_path = 'labeled_data_RougeL.json' #output file name
save_labels(labeled_data, output_file_path)

print(f"Labeled data has been saved to {output_file_path}")

Labeled data has been saved to labeled_data.json
